### 1.a. Investigate and explain the Bellman-Ford algorithm to calculate all shortest paths between a given node v and all other nodes w in a weighted network. Implement the algorithm in python and test your method in the example network from the theory lecture?

The Bellman Ford algorithm works in three steps:
* initialization
* searching for paths
* checking for cycle with negative weights
The search step in algorithm works with that fact that in the worst case the algorithm has the most far node in n - 1 steps. So the algorithm works with two cycle. The first one iterate from 1 to n - 1 cases. The second cycle each time iterate through the all edges and check if the edge can change the value of the shortest path of the connected nodes.

In [ ]:
import pathpy as pp

n = pp.Network(directed=True)
n.add_edges(
    pp.Edge('f','g', weight=2, uid='f-g'),
    pp.Edge('g','d', weight=2, uid='g-d'),
    pp.Edge('f','d', weight=3, uid='f-d'),
    pp.Edge('f','e', weight=1, uid='f-e'),
    pp.Edge('e','f', weight=1, uid='e-f'),
    pp.Edge('d','e', weight=2, uid='d-e'),
    pp.Edge('b','d', weight=1, uid='b-d'),
    pp.Edge('b','c', weight=2, uid='b-c'),
    pp.Edge('a','b', weight=1, uid='a-b'),
    pp.Edge('c','a', weight=1, uid='c-a')
    )

n.plot(node_color="red")

In [ ]:
from typing import Dict, Any
from copy import copy
from math import inf

def bellman_form_search_iteration(network: pp.Network, shortest_paths: Dict[str, float]) -> Dict[str, float]:

    for _ in range(len(network.nodes) - 1):
        for edge in network.edges:
            left_node_id, right_node_id = edge.nodes.keys()

            if not network.directed and shortest_paths[left_node_id] > shortest_paths[right_node_id]:
                left_node_id, right_node_id = right_node_id, left_node_id

            if (new_value := shortest_paths[left_node_id] + edge.weight()) < shortest_paths[right_node_id]:
                shortest_paths[right_node_id] = new_value
    return shortest_paths


def bellman_ford_nearest_routes(network: pp.Network, start_node_id: str) -> Dict[str, Any]:
    shortest_paths = {}

    # init the start point
    for node in network.nodes:
        if node.uid == start_node_id:
            value = 0
        else:
            value = inf
        shortest_paths[node.uid] = value

    # Find the paths
    result = bellman_form_search_iteration(network, shortest_paths)

    # Check the cycles with edge wights less than 0
    result2 = bellman_form_search_iteration(network, copy(result))

    if result2 != result:
        result = {"Error": "The graph with cycle with edges weight less 0!"}

    return result

bellman_ford_nearest_routes(n, 'g')

### 1.b. Develop an algorithm that uses the powers of adjacency matrix to calculate the diameter of a directed network. You can assume that the network is connected i.e. your algorithm does not need to terminate if the network is disconnected. Implement your algorithm in python and test it in a directed network e.g. using the software package pathpy?

There is a property of representing graphs as adjacency matrix, which is to continously take powers of the matrix to see when the last zero entry disappears. At that point, we can deduce the diameter of the matrix.

In [ ]:
import pathpy as pp

n = pp.Network(directed=False)
n.add_edges(
    pp.Edge('f','g'),
    pp.Edge('g','d'),
    pp.Edge('f','d'),
    pp.Edge('f','e'),
    pp.Edge('d','e'),
    pp.Edge('b','d'),
    pp.Edge('b','c'),
    pp.Edge('a','b'),
    pp.Edge('c','a')
    )

n.plot(node_color="red")

With the assumption that the algorithm does not need to terminate if the network is disconnected, such a function can be written to find the diamater of a connected network:

In [ ]:
import numpy as np

def find_diameter(n: pp.Network):
    adj_matrix = n.adjacency_matrix().todense()
    i = 0

    while not np.all(adj_matrix):
        i += 1
        adj_matrix += pow(adj_matrix, i)
    return i

print(find_diameter(n))
print(pp.algorithms.shortest_paths.diameter(n))

## 2. Answer the following questions about the partition quality measure Q(G, C) that was introduced in the lecture 2?

### a. Consider a fully-connected(i.e. all links exists) and undirected network G = (V, E) with n nodes and no self-loops. Further, assume that all nodes are assigned to a single community i.e. consider a partition C = {V}. Prove that Q(G, C) = 0?

We will use formula:

![Partition quality formula](images/2_qgc_formula.jpg)

For the n cases:
(1/2m) * n * (0 - ((n-1) ^ 2) / 2m)

For n * (n-1) cases:
(1/2m) * n * (n - 1) * (1 - ((n - 1) ^ 2)/2m)

As the result formula goes to 1 / 2n which is actually 1 / inf -> 0

### b. Consider an undirected network G = (V, E) that exclusively contains self-loops. Assume that self-loops are represented by a one-entry on the main diagonal of the adjacency matrix i.e. A = diag(1;....1). Consider a community partition C, where all nodes are assigned to different communities i.e. C = {{v1}, ...., {vn}} for V = {v1, ..., vn}. Prove that Q(G, C) -> 1/2 for n -> inf?

We will use formula:

![Partition quality formula](images/2_qgc_formula.jpg)

The formula is OK only for n cases of self-loops.
In a such case the formula is equal to (1/2n) * n * (1 - 1/2n).
This leads us to lim(1/2 - 1/4n) where n -> inf. It means that lim(1/2 - 0) = 1/2.

### 3.a. Construct a network in which the node with the highest betweenness centrality has the smallest degree centrality. Use pathpy to demonstrate the correctness of your example?

Node A in the graph below, has the following properties:
- Highest betweenness centrality (Because any given path between two random nodes has a high chance to pass through Node A
- Smallest degree centrality (All nodes are highly connected to each other, making Node A's centrality smaller)

In [ ]:
import pathpy as pp

n = pp.Network(directed=False)
n.add_edges(
    pp.Edge('a','b'),
    pp.Edge('a','c'),
    pp.Edge('b','d'),
    pp.Edge('b','e'),
    pp.Edge('e','f'),
    pp.Edge('f','d'),
    pp.Edge('e','d'),
    pp.Edge('b','f'),
    pp.Edge('c','g'),
    pp.Edge('c','h'),
    pp.Edge('c','i'),
    pp.Edge('h','i'),
    pp.Edge('g','i'),
    pp.Edge('h','g'),
    )

n.plot(node_color="red")

With the code below, we see that Node a indeed has this property.

In [ ]:
import operator

print(min(pp.algorithms.centralities.degree_centrality(n)) ==
      max(pp.algorithms.centralities.betweenness_centrality(n).items(),
          key=operator.itemgetter(1))[0])

### 3.b. Construct a network in which with 10 nodes where exactly one node has the max possible closeness centrality?

In [ ]:
import pathpy as pp

n = pp.Network(directed=False)
n.add_edges(
    pp.Edge('a','b'),
    pp.Edge('a','c'),
    pp.Edge('a','d'),
    pp.Edge('a','e'),
    pp.Edge('a','f'),
    )

n.plot(node_color="red")

In [ ]:
import operator

print(max(pp.algorithms.centralities.betweenness_centrality(n).items(),
          key=operator.itemgetter(1))[0])

### 3.c Give an example for a network with 10 nodes where exactly one node has the max betweenness centrality possible in a network with that size. Prove that the max possible betweenness centrality in a network with n nodes is $n^2$ - 2n - n + 2?

We will use such a formula:

![alt text](images/3_c_betweenness_centrality_formula.jpg)

Nst(v) / Nst > 0 for all cases. We want to maximize the sum. For that, Nst(v) / Nst should be 1, since its max value is 1 since Nst(v) cannot be higher than Nst. Let's take one node for calculating betweeness centrality. For any given node from which we construct all the shortest paths, the max occurs when there is just one node from which all the possible shortest paths has to pass. (which is the case in the example graph we constructed previously for n=10). We are considering all the shortest paths sourced from any given node. We have n-1 ways of choosing the source node. We have n-2 ways for choosing the destination node. Note that when calculating the sum for v: s,t ∈ - {v} must hold by the formula. So, (n-1)(n-2) = $n^2$ - 2n - n + 3 = $n^2$ -3n + 3.

Cb(Node A) = 72 and (10-1)(10-2) = 72 as well.